In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import matplotlib.pyplot as plt
import numpy as np
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode
import os, json, cv2, random
from numpy import load
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode

import pyrealsense2
from realsense_depth import *
import time

In [ ]:
#NumpyarrPath = "/content/drive/MyDrive/UnivJobData/augmentedDATA.npz"
JSONpath = "../NewFinals/augmentations/Level0/augmentedJSON.json"  ##Argument 1
classes = ["RockerArm","BoltHoles","Big_PushRodHoles","Small_PushRodHoles", "Engine", "Bolt","PushRod","RockerArmObject"]  ##Argument 2
imgPath = "../NewFinals/augmentations/Level0/"  ##Argument 3
programName = 'engine_'  ##Argument 4
hasKeypoints = True  ##Argument 5
hasSegmentationAndBBox = True  ##Argument 6
onlyBBox = False  ##Argument 7

if hasKeypoints == True:
    maxNumOfKeypoints = 25  ## give only if your dataset has keypoints   ##Argument 8
    kpString = []
    for i in range(1,maxNumOfKeypoints+1):
        kpWord = "kp"+str(i)
        kpString.append(kpWord)
with open(JSONpath) as f:
    imgs_anns = json.load(f)

In [ ]:
#def get_dicts(Images,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
def get_dicts(img_dir,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox):
    dataset_dicts = []
    for idx, v in enumerate(imgs_anns):
      record = {}
      filename = os.path.join(img_dir, v["image"])
      height, width = cv2.imread(filename).shape[:2]
      record["file_name"] = filename
      record["image_id"] = idx
      record["height"] = height
      record["width"] = width

      #### code for key points start ###
      if ("kp-1" in v) and (hasKeypoints == True):
        kps = v["kp-1"]
        obkps=[]
        countOfKeypoints = len(kps)
        for _, kp in enumerate(kps):
          x_kp = kp["x"] * width/100
          y_kp = kp["y"] * height/100
          v_kp = 1 #look at keypoints, it is not visible but labeled
          obkps.append(x_kp)
          obkps.append(y_kp)
          obkps.append(v_kp)
        
        if countOfKeypoints < maxNumOfKeypoints:
          for kp in range(0,(maxNumOfKeypoints-countOfKeypoints)):
            obkps.append(0)
            obkps.append(0)
            obkps.append(0)
      elif hasKeypoints == True:  ## Remove if your dataset does not have keypoints
        obkps=[]
        for kp in range(0,maxNumOfKeypoints):
          obkps.append(0)
          obkps.append(0)
          obkps.append(0)
        #### code for key points ends ###
      if "label" in v and (hasSegmentationAndBBox == True):
        annos = v["label"]
      objs = []
      noSegmentationPoints = 0
      for _, anno in enumerate(annos):
        if "label" in v:              
          getPoints = np.array(anno["points"])
          if len(getPoints) != 0:
            getPoints[:,0] = getPoints[:,0] * width/100
            getPoints[:,1] = getPoints[:,1] * height/100
            px = getPoints[:,0]
            py = getPoints[:,1]
            poly = (getPoints.flatten()).tolist()
          else:
            noSegmentationPoints = 1   ## If any image does not contains segmentations

        for ind in range(0,len(classes)):
          if anno["polygonlabels"][0] == classes[ind]:
            catId= ind        
        if ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 0):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": obkps,
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and (onlyBBox == False) and (hasSegmentationAndBBox == True):
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [poly],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("label" in v) and (noSegmentationPoints == 0) and onlyBBox == True:
          obj = {
              "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
              "bbox_mode": BoxMode.XYXY_ABS,
              "segmentation": [],
              "category_id": catId,
              "keypoints": [],
          }
        elif ("kp-1" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": [],
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": []
          }
        elif ("kp-1" in v) and ("label" in v) and (noSegmentationPoints == 1):
          obj = {
              "keypoints": obkps,
              "segmentation": [],
              "bbox": BoxMode.XYXY_ABS,
              "bbox_mode": BoxMode.XYXY_ABS,
              "category_id": catId
          }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


for d in ["val"]:
    DatasetCatalog.register(programName + d, lambda d=d: get_dicts(imgPath,imgs_anns,hasKeypoints,hasSegmentationAndBBox,onlyBBox))
    MetadataCatalog.get(programName + d).set(thing_classes=classes)
    if hasKeypoints == True:
        MetadataCatalog.get(programName + d).set(keypoint_names = kpString)
        MetadataCatalog.get(programName + d).set(keypoint_flip_map = [])
engine_metadata = MetadataCatalog.get(programName+"val")

In [ ]:
cfg = get_cfg()
cfg.MODEL.MASK_ON = True
cfg.MODEL.KEYPOINT_ON = False
cfg.MODEL.ROI_KEYPOINT_HEAD.NORMALIZE_LOSS_BY_VISIBLE_KEYPOINTS = True

cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 # Set threshold for this model
#cfg.DATASETS.TEST = ("engine_val",)
cfg.MODEL.WEIGHTS = '../Dataset/output_level1/model_final.pth' # Set path model .pth
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 8
cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = maxNumOfKeypoints
predictor = DefaultPredictor(cfg)

#import the COCO Evaluator to use the COCO Metrics
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

#Call the COCO Evaluator function and pass the Validation Dataset
evaluator = COCOEvaluator("engine_val", cfg, False, output_dir="../Dataset/")
val_loader = build_detection_test_loader(cfg, "engine_val")

#Use the created predicted model in the previous step
inference_on_dataset(predictor.model, val_loader, evaluator)

In [ ]:
import cv2
import numpy as np

all_camera_idx_available = []

for camera_idx in range(-2,10):
    print(camera_idx)
    cap = cv2.VideoCapture(camera_idx)
    if cap.isOpened():
        print(camera_idx)
        print(f'Camera index available: {camera_idx}')
        all_camera_idx_available.append(camera_idx)
        cap.release()

In [ ]:
for d in ["train"]:
    MetadataCatalog.get(programName + d).set(thing_classes=classes)
    MetadataCatalog.get(programName + d).set(keypoint_names = kpString)
    MetadataCatalog.get(programName + d).set(keypoint_flip_map = [])
engine_metadata = MetadataCatalog.get(programName+"_train")

In [ ]:
from numpy import asarray
from numpy import savez_compressed
import time
cap = cv2.VideoCapture(4)

if not (cap.isOpened()):
    print("Could not open video device")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,720)
prev_frame_time = time.time()

count = 0
while(True):    
    # Capture frame-by-frame    
    
    ret, frame = cap.read()
    outputs = predictor(frame)
    
    v = Visualizer(frame[:, :, ::-1],
                   metadata=engine_metadata,
                   #MetadataCatalog.get(cfg.DATASETS.TRAIN[0]),
                   scale=1,
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2.imshow("prediction",v.get_image()[:, :, ::-1])
    #cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):    
        break
    
    new_frame_time = time.time()
    seconds = new_frame_time - prev_frame_time
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    #cv2.imwrite("frame720_720.jpg", frame)
    #print("fps",fps)
    #Waits for a user input to quit the application    
    count+=1
    
    #if count == 25:
    #    segMask = outputs["instances"].to("cpu").pred_masks.numpy()
    #    img = frame
    #    print(segMask.shape)
    #    print(img.shape)
    #    savez_compressed('../samples/sample5segMask.npz', segMask)
    #    savez_compressed('../samples/sample5img.npz', img)
cap.release()
cv2.destroyAllWindows()